<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
# Imports

import pandas as pd
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [2]:
# Import Amazon Consumers Review csv

df = pd.read_csv('../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [3]:
# Check out the head of the Amazon Consumer Reviews dataframe

df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [4]:
# Set stopwords

STOPWORDS = set(STOPWORDS).union(set(['amazon', 'amazonbasics']))

# Define tokenizer function

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [5]:
# Tokenize reviews.text column 

df['tokens'] = df['reviews.text'].apply(lambda x: tokenize(x))

In [6]:
# A Dictionary Representation of all the words in our corpus

id2word = corpora.Dictionary(df['tokens'])

In [7]:
# Check out list of 20 id2word items

list(id2word.items())[:20]

[(0, 'aluminum'),
 (1, 'backup'),
 (2, 'bad'),
 (3, 'battery'),
 (4, 'item'),
 (5, 'missing'),
 (6, 'order'),
 (7, 'pcs'),
 (8, 'quality'),
 (9, 'spring'),
 (10, 'work'),
 (11, 'bulk'),
 (12, 'expensive'),
 (13, 'like'),
 (14, 'products'),
 (15, 'way'),
 (16, 'duracell'),
 (17, 'happy'),
 (18, 'price'),
 (19, 'batteries')]

In [9]:
# Make a corpus with bag of words

# A bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the tokenize f(x)

corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [10]:
# Make a LDA Topic Model

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15, # depends on the corpus
                   passes=10,
                   workers=12
                  )

# sklearn does all the computations right away, but gensim does not do this, it uses the print_topics() method

In [11]:
# Print LDA topics

lda.print_topics()

[(0,
  '0.031*"daughter" + 0.021*"great" + 0.020*"kids" + 0.018*"easy" + 0.017*"loves" + 0.014*"gift" + 0.014*"purchased" + 0.013*"love" + 0.011*"learning" + 0.011*"use"'),
 (1,
  '0.044*"tablet" + 0.034*"games" + 0.027*"loves" + 0.024*"apps" + 0.020*"kids" + 0.019*"old" + 0.018*"year" + 0.017*"great" + 0.015*"daughter" + 0.015*"love"'),
 (2,
  '0.036*"love" + 0.024*"tablet" + 0.020*"bought" + 0.018*"kindle" + 0.015*"screen" + 0.011*"great" + 0.008*"apps" + 0.008*"grandkids" + 0.008*"hd" + 0.008*"wife"'),
 (3,
  '0.068*"great" + 0.043*"bought" + 0.040*"loves" + 0.040*"use" + 0.033*"easy" + 0.028*"old" + 0.028*"gift" + 0.025*"tablet" + 0.023*"year" + 0.021*"christmas"'),
 (4,
  '0.032*"old" + 0.024*"tablet" + 0.021*"battery" + 0.019*"use" + 0.016*"kindle" + 0.015*"great" + 0.015*"year" + 0.015*"like" + 0.012*"purchased" + 0.012*"life"'),
 (5,
  '0.038*"great" + 0.030*"tablet" + 0.027*"works" + 0.020*"price" + 0.018*"apps" + 0.016*"google" + 0.014*"buy" + 0.012*"good" + 0.012*"play" + 0.

In [12]:
# Get words from lda.print_topics, whicle cleaning them when regex

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [13]:
# Join words to form topics

topics = [' '.join(t[0:5]) for t in words]

In [14]:
# Print Topics

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")
    
# difficult with similar topics, yet these seem good for gift shopping and marketing

------ Topic 0 ------
daughter great kids easy loves

------ Topic 1 ------
tablet games loves apps kids

------ Topic 2 ------
love tablet bought kindle screen

------ Topic 3 ------
great bought loves use easy

------ Topic 4 ------
old tablet battery use kindle

------ Topic 5 ------
great tablet works price apps

------ Topic 6 ------
tablet movies great watch use

------ Topic 7 ------
tablet great price good value

------ Topic 8 ------
batteries brand work like buy

------ Topic 9 ------
kids tablet great love time

------ Topic 10 ------
tablet good work bought buy

------ Topic 11 ------
screen kindle great little nice

------ Topic 12 ------
kindle easy books read use

------ Topic 13 ------
tablet good buy bought best

------ Topic 14 ------
good great price batteries long



In [15]:
# Import pyLDAvis for awesome visualtions

### THIS WILL BREAK JUPYTER LAB BUTTONS, JUPYTER NOTEBOOK IS OKAY, ELSEWHERE USE AT OWN RISK

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [16]:
# pyLDAvis Gensim visualization of LSA, Corpus, and id2word

pyLDAvis.gensim.prepare(lda, corpus, id2word)

# Put slider close to 0, hover over a topic, and it will show terms that are more unique to that topic

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.220324  0.000406       1        1  16.721869
14    -0.250824 -0.065011       2        1  10.132047
1      0.162219 -0.095958       3        1   8.997715
3      0.104227 -0.107576       4        1   8.302979
12     0.112550  0.160574       5        1   7.650992
9      0.067738 -0.084506       6        1   5.900613
11     0.001931  0.167277       7        1   5.519578
7     -0.055806 -0.027650       8        1   5.310150
4      0.039845  0.012836       9        1   5.236845
6     -0.010079  0.025988      10        1   5.054893
2      0.065818  0.040565      11        1   4.988061
13    -0.020597  0.038437      12        1   4.799358
10    -0.037748 -0.036936      13        1   4.723277
5     -0.034298  0.045887      14        1   4.317001
0      0.075347 -0.074335      15        1   2.344615, topic_info=           Term         Freq        Total Category  logprob  loglift
19    batteries  6733.000000  6733.000000  Default  30.0000  30.0000
4194     tablet  7162.000000  7162.000000  Default  29.0000  29.0000
200        kids  2528.000000  2528.000000  Default  28.0000  28.0000
33         good  5815.000000  5815.000000  Default  27.0000  27.0000
2614      loves  2103.000000  2103.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
33         good    48.823273  5815.458008  Topic15  -5.0189  -1.0270
27       bought    42.361275  3071.001465  Topic15  -5.1608  -0.5305
233        best    39.912395  1483.688477  Topic15  -5.2204   0.1374
541      little    37.423126  1002.819336  Topic15  -5.2848   0.4648
3154     kindle    38.046959  3178.316406  Topic15  -5.2683  -0.6722

[1246 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
56        1  0.822684    aa
56        2  0.163628    aa
56        6  0.002273    aa
56        9  0.002273    aa
56       11  0.002273    aa
...     ...       ...   ...
6328      4  0.038464   äôt
6328      7  0.057696   äôt
6328     12  0.826975   äôt
6328     15  0.019232   äôt
8920     12  0.880609  äôve

[4513 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 15, 2, 4, 13, 10, 12, 8, 5, 7, 3, 14, 11, 6, 1])

In [17]:
# LDA corpus at first position, to begin scoring topic distribution

lda[corpus[0]]

[(5, 0.11362065), (6, 0.56039065), (13, 0.25917262)]

In [18]:
# Make topic distribution

distro = [lda[d] for d in corpus]

In [19]:
# Verify distribution is in order

distro[0]

[(0, 0.11201345), (6, 0.58708197), (13, 0.23409508)]

In [20]:
# Update the distribution to a complete distribution

distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [21]:
# Check length of new distribution

len(new_distro)

28332

In [22]:
# Check df head

df.head()

# tokens are at the end, consider making a smaller df with less columns

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokens
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracell, price, happy]"
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteries, better, price]"
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteries, long, lasting, price, great]"


In [28]:
# Make a new dataframe called tf for topicframe, for the distribution

tf = pd.DataFrame.from_records(new_distro, index=df.primaryCategories)
tf.columns = topics

In [29]:
# Check out the head of the tf

tf.head()

,daughter great kids easy loves,tablet games loves apps kids,love tablet bought kindle screen,great bought loves use easy,old tablet battery use kindle,great tablet works price apps,tablet movies great watch use,tablet great price good value,batteries brand work like buy,kids tablet great love time,tablet good work bought buy,screen kindle great little nice,kindle easy books read use,tablet good buy bought best,good great price batteries long
primaryCategories,,,,,,,,,,,,,,,
Health & Beauty,0.000000,0.000000,0.000000,0.000000,0.000000,0.114951,0.452744,0.000000,0.155610,0.000000,0.000000,0.000000,0.000000,0.215417,0.000000
Health & Beauty,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.844438,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112
Health & Beauty,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766661
Health & Beauty,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844443,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111
Health & Beauty,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844444


In [32]:
# Make a Coherence Model

from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

# higher coherence means when the topics were taken, words samlpled according to proportions, would recreate the document really well
# complex, but good to have intuition, and study more

In [ ]:
# Declare coherence values

# Can take a long time to run.

coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=5) # don't want this number to be low, because then the model won't learn, too high and wastes computatinal resources

In [ ]:
# Make topic coherence dataframe

topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
# Check out head of topic coherence dataframe

topic_coherence.head()

In [ ]:
# Plot topic coherence

import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

# Don't want topics too low, or too high, maybe around 9 in lecture example, then go back and retrain LDA model on this number
# Go back and rename topics with human intuition, not easy to do, but useful
# What are topics useful for? ie: use topics for recommendations

In [ ]:
# Print the coherence scores

for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Enter a sample document to score with a topic distribution

lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling